In [1]:
import os, re, time, json, math
import datetime
import PIL.Image, PIL.ImageFont, PIL.ImageDraw
import numpy as np
import tensorflow as tf
import pandas as pd
import google
from matplotlib import pyplot as plt
print("Tensorflow version " + tf.__version__)

Tensorflow version 2.3.1


In [30]:
import tensorflow as tf
import tensorflow.keras as keras

from src.generators.tf_records_generator import get_dataset, count_data_items
from src.models.efficientNet_B0 import EffNet0, freeze_blocks
from src.utils.log_files import save_config, save_fold_iter_history, get_log_dir, save_logs_pickle, generate_columns
from src.utils.kfold import get_kfold_split
from src.utils.schedules import CLR_schedule
from clr_schedule import CyclicLR

ImportError: cannot import name 'save_logs_pickle' from 'src.utils.log_files' (D:\inzynierka\src\utils\log_files.py)

In [29]:
CONFIG = dict(
    log_prefix = "CLR_",
    effnet_version = 0,
    input_shape=(256, 256, 3),
    image_resolution=256,
    trainable_base=False,
    time=datetime.datetime.now().strftime("_%d_%m_%Y_%H_%M"),
    use_patient_data=True,
    inner_blocks_frozen=6,

    lr_min=0.0001,
    lr_max=0.01,
    lr_decay=None,
    
    replicas=1,
    steps_per_epoch=None,
    validation_steps=None,
    batch_size=8,
    epochs=100,
    optimizer='adam',
    loss='binary_crossentropy',
    
    output_bias=np.log([584/32542]),
    weight_for_0 = (1 / 32542)*(32542+584)/2.0,
    weight_for_1 = (1 / 584)*(32542+584)/2.0
)

0.9120108393559098


In [4]:
IS_COLAB_BACKEND = 'COLAB_GPU' in os.environ  # this is always set on Colab, the value is 0 or 1 depending on GPU presence
if IS_COLAB_BACKEND:
  from google.colab import auth
  # Authenticates the Colab machine and also the TPU using your
  # credentials so that they can access your private GCS buckets.
  auth.authenticate_user()
  data_dir = 'gs://dataset_files/'
else:
  data_dir = 'dataset/'

In [5]:
# Detect hardware
try:
  tpu_resolver = tf.distribute.cluster_resolver.TPUClusterResolver() # TPU detection
except ValueError:
  tpu_resolver = None
  gpus = tf.config.experimental.list_logical_devices("GPU")

# Select appropriate distribution strategy
if tpu_resolver:
  tf.config.experimental_connect_to_cluster(tpu_resolver)
  tf.tpu.experimental.initialize_tpu_system(tpu_resolver)
  strategy = tf.distribute.TPUStrategy(tpu_resolver)
  print('Running on TPU ', tpu_resolver.cluster_spec().as_dict()['worker'])
elif len(gpus) > 1:
  strategy = tf.distribute.MirroredStrategy([gpu.name for gpu in gpus])
  print('Running on multiple GPUs ', [gpu.name for gpu in gpus])
elif len(gpus) == 1:
  strategy = tf.distribute.get_strategy() # default strategy that works on CPU and single GPU
  print('Running on single GPU ', gpus[0].name)
else:
  strategy = tf.distribute.get_strategy() # default strategy that works on CPU and single GPU
  print('Running on CPU')
  
print("Number of accelerators: ", strategy.num_replicas_in_sync)
CONFIG["replicas"] = strategy.num_replicas_in_sync


Running on single GPU  /device:GPU:0
Number of accelerators:  1


In [6]:
path = data_dir + str(CONFIG["image_resolution"])
files_train = np.sort(np.array(tf.io.gfile.glob(path + '/train*.tfrec')))
files_test = np.sort(np.array(tf.io.gfile.glob(path + '/test*.tfrec')))


ds = get_dataset(files_train[:-3], CONFIG)
val_ds = get_dataset(files_train[-3], CONFIG)
test_ds = get_dataset(files_test, CONFIG)

In [10]:
def make_model(config:dict, strategy)    :
    with strategy.scope():
      model = EffNet0(config=config, trainable_base=config['trainable_base'])

      model.compile(optimizer=keras.optimizers.Adam(), loss=config["loss"], 
                    metrics=[
                                  keras.metrics.TruePositives(name='tp'),
                                  keras.metrics.FalsePositives(name='fp'),
                                  keras.metrics.TrueNegatives(name='tn'),
                                  keras.metrics.FalseNegatives(name='fn'), 
                                  keras.metrics.BinaryAccuracy(name='accuracy'),
                                  keras.metrics.Precision(name='precision'),
                                  keras.metrics.Recall(name='recall'),
                                  keras.metrics.AUC(name='auc'),
                            ]
    )
    return model


In [11]:
kfold_split = get_kfold_split(files_train)
save_config(CONFIG)
logs = pd.DataFrame()
for iter, files in kfold_split.items():
    train_ds = get_dataset(files['train'],CONFIG, patient_info=True,repeat=True) 
    val_ds = get_dataset(files['validation'], CONFIG, patient_info=True,repeat=True)
    if CONFIG['steps_per_epoch'] is None:
        CONFIG['steps_per_epoch'] = math.ceil(count_data_items(files["train"])/(CONFIG["batch_size"]*CONFIG["replicas"]))
    if CONFIG['validation_steps'] is None:
        validation_steps = math.ceil(count_data_items(files["validation"])/(CONFIG["batch_size"]*CONFIG["replicas"]))
    
    lr = CyclicLR(base_lr=CONFIG["lr_min"], max_lr=CONFIG["lr_max"],
                                step_size=CONFIG['steps_per_epoch']*4, mode='triangular')
    
    model = make_model(CONFIG, strategy=strategy)
    history = model.fit(train_ds,class_weight={0:CONFIG["weight_for_0"],1:CONFIG["weight_for_1"]},
                        validation_data=val_ds,validation_batch_size=CONFIG["batch_size"]*CONFIG["replicas"],
                        batch_size=CONFIG["batch_size"]*CONFIG["replicas"], epochs=CONFIG["epochs"],
                        steps_per_epoch=CONFIG['steps_per_epoch'],validation_steps=CONFIG['validation_steps'])
    
    if logs.empty:
        logs = generate_columns(logs,history.history)
    logs = logs.append(history.history, ignore_index=True)
    
save_logs_pickle(logs,CONFIG)



0:		1e-10

Epoch 00001: LearningRateScheduler reducing learning rate to 1e-10.
Epoch 1/21
20/20 [==============================] - 5s 241ms/step - loss: 2.3021 - tp: 0.0000e+00 - fp: 0.0000e+00 - tn: 157.0000 - fn: 3.0000 - accuracy: 0.9812 - precision: 0.0000e+00 - recall: 0.0000e+00 - auc: 0.3344 - val_loss: 0.1052 - val_tp: 0.0000e+00 - val_fp: 0.0000e+00 - val_tn: 157.0000 - val_fn: 3.0000 - val_accuracy: 0.9812 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00 - val_auc: 0.4724

In [15]:
log_dir = get_log_dir(CONFIG)
zip_name = os.path.basename(log_dir)
!zip -r $zip_name $log_dir

  adding: logs/TL_EffN0_256_02_11_2020_21_13/ (stored 0%)
  adding: logs/TL_EffN0_256_02_11_2020_21_13/0.json (deflated 52%)
  adding: logs/TL_EffN0_256_02_11_2020_21_13/config.json (deflated 33%)
  adding: logs/TL_EffN0_256_02_11_2020_21_13/1.json (deflated 53%)
